# 第4章: マルチエージェント・ワークフロー

## 準備

以下のセルを順番に実行して、演習に必要な環境をセットアップします。

### ライブラリのインストール

このセルは、LangGraphおよび関連するLangChainライブラリをインストールします。実行には数分かかる場合があります。
ご利用になるLLMプロバイダーに応じて、コメントアウトを解除して必要なライブラリをインストールしてください。

In [ ]:
# === ライブラリのインストール ===
# 基本ライブラリ (LangGraphとLangChain Core)
!%pip install -qU langchain langgraph

# --- LLMプロバイダー別ライブラリ ---
# ご利用になるLLMプロバイダーに応じて、以下の該当する行のコメントを解除して実行してください。

# OpenAI (GPTシリーズ)
# !%pip install -qU langchain_openai

# Azure OpenAI
# !%pip install -qU langchain_openai # Azureもlangchain_openaiを利用

# Google Cloud Vertex AI (Gemini, PaLM等)
# !%pip install -qU langchain_google_vertexai

# Google Gemini API (Google AI Studioで利用するGemini)
# !%pip install -qU langchain_google_genai

# Anthropic (Claudeシリーズ)
# !%pip install -qU langchain_anthropic

# Amazon Bedrock (AWS上の各種モデル、Claudeも含む)
# !%pip install -qU langchain_aws boto3 # Bedrock利用時はboto3も必要

# --- その他の推奨ライブラリ ---
# グラフの可視化や環境変数管理など、演習全体を通して利用する可能性のあるライブラリ
!%pip install -qU python-dotenv pygraphviz pydotplus graphviz

### LLMプロバイダーの選択

このセルでは、使用するLLMプロバイダーを選択します。
`LLM_PROVIDER` 変数に、利用したいプロバイダー名を設定してください。
選択可能なプロバイダー: `"openai"`, `"azure"`, `"google"` (Vertex AI), `"google_genai"` (Gemini API), `"anthropic"`, `"bedrock"`

In [ ]:
# === LLMプロバイダーの選択 ===
# 利用したいLLMプロバイダーを以下の変数で指定してください。
# "openai", "azure", "google" (Vertex AI), "google_genai" (Gemini API), "anthropic", "bedrock" のいずれかを選択できます。
LLM_PROVIDER = "openai"  # 例: OpenAI を利用する場合

### APIキー/環境変数の設定

以下のセルを実行する前に、選択したLLMプロバイダーに応じたAPIキーまたは環境変数を設定する必要があります。

**一般的な手順:**
1.  `.env.sample` ファイルをコピーして `.env` ファイルを作成します。
2.  `.env` ファイルを開き、選択したLLMプロバイダーに対応するAPIキーや必要な情報を記述します。
    *   **OpenAI:** `OPENAI_API_KEY`
    *   **Azure OpenAI:** `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT`, `OPENAI_API_VERSION`, `AZURE_OPENAI_DEPLOYMENT_NAME`
    *   **Google (Vertex AI):** `GOOGLE_CLOUD_PROJECT_ID`, `GOOGLE_CLOUD_LOCATION` (Colab環境外で実行する場合、`GOOGLE_APPLICATION_CREDENTIALS` 環境変数の設定も必要になることがあります)
    *   **Google (Gemini API):** `GOOGLE_API_KEY`
    *   **Anthropic:** `ANTHROPIC_API_KEY`
    *   **AWS Bedrock:** `AWS_ACCESS_KEY_ID`, `AWS_SECRET_ACCESS_KEY`, `AWS_REGION_NAME` (IAMロールを使用する場合は、これらのキー設定は不要な場合がありますが、リージョン名は必須です)
3.  ファイルを保存します。

**Google Colab を使用している場合:**
上記の `.env` ファイルを使用する代わりに、Colabのシークレットマネージャーに必要なキーを登録してください。
例えば、OpenAIを使用する場合は `OPENAI_API_KEY` という名前でシークレットを登録します。
Vertex AI を利用する場合は、Colab上での認証 (`google.colab.auth.authenticate_user()`) が実行されます。

このセルは、設定された情報に基づいて環境変数をロードし、LLMクライアントを初期化します。

In [ ]:
# === APIキー/環境変数の設定 ===
import os
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む (存在する場合)
load_dotenv()

try:
    from google.colab import userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

# --- OpenAI ---
if LLM_PROVIDER == "openai":
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY and IS_COLAB:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 OPENAI_API_KEY を設定するか、Colab環境の場合はシークレットに OPENAI_API_KEY を設定してください。")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# --- Azure OpenAI ---
elif LLM_PROVIDER == "azure":
    AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
    AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
    OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
    AZURE_OPENAI_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if IS_COLAB:
        if not AZURE_OPENAI_API_KEY: AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
        if not AZURE_OPENAI_ENDPOINT: AZURE_OPENAI_ENDPOINT = userdata.get("AZURE_OPENAI_ENDPOINT")
        if not OPENAI_API_VERSION: OPENAI_API_VERSION = userdata.get("OPENAI_API_VERSION") # 例: "2023-07-01-preview"
        if not AZURE_OPENAI_DEPLOYMENT_NAME: AZURE_OPENAI_DEPLOYMENT_NAME = userdata.get("AZURE_OPENAI_DEPLOYMENT_NAME")

    if not AZURE_OPENAI_API_KEY: raise ValueError("Azure OpenAI APIキー (AZURE_OPENAI_API_KEY) が設定されていません。")
    if not AZURE_OPENAI_ENDPOINT: raise ValueError("Azure OpenAI エンドポイント (AZURE_OPENAI_ENDPOINT) が設定されていません。")
    if not OPENAI_API_VERSION: OPENAI_API_VERSION = "2023-07-01-preview" # デフォルトを設定することも可能
    if not AZURE_OPENAI_DEPLOYMENT_NAME: raise ValueError("Azure OpenAI デプロイメント名 (AZURE_OPENAI_DEPLOYMENT_NAME) が設定されていません。")

    os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY
    os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
    os.environ["OPENAI_API_VERSION"] = OPENAI_API_VERSION

# --- Google Cloud Vertex AI (Gemini) ---
elif LLM_PROVIDER == "google":
    PROJECT_ID = os.environ.get("GOOGLE_CLOUD_PROJECT_ID") # .env 用に修正
    LOCATION = os.environ.get("GOOGLE_CLOUD_LOCATION")

    if IS_COLAB:
        if not PROJECT_ID: PROJECT_ID = userdata.get("GOOGLE_CLOUD_PROJECT_ID")
        if not LOCATION: LOCATION = userdata.get("GOOGLE_CLOUD_LOCATION") # 例: "us-central1"
        from google.colab import auth as google_auth
        google_auth.authenticate_user() # Vertex AI を使う場合は Colab での認証を推奨
    else: # Colab外の場合、.envから読み込んだ値で環境変数を設定
        if PROJECT_ID: os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID # Vertex AI SDKが参照する標準的な環境変数名
        if LOCATION: os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION

    if not PROJECT_ID: raise ValueError("Google Cloud Project ID が設定されていません。環境変数 GOOGLE_CLOUD_PROJECT_ID を設定するか、Colab環境の場合はシークレットに GOOGLE_CLOUD_PROJECT_ID を設定してください。")
    if not LOCATION: LOCATION = "us-central1" # デフォルトロケーション

# --- Google Gemini API (langchain-google-genai) ---
elif LLM_PROVIDER == "google_genai":
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY and IS_COLAB:
        GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
    if not GOOGLE_API_KEY:
        raise ValueError("Google APIキーが設定されていません。環境変数 GOOGLE_API_KEY を設定するか、Colab環境の場合はシークレットに GOOGLE_API_KEY を設定してください。")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# --- Anthropic (Claude) ---
elif LLM_PROVIDER == "anthropic":
    ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY and IS_COLAB:
        ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("Anthropic APIキーが設定されていません。環境変数 ANTHROPIC_API_KEY を設定するか、Colab環境の場合はシークレットに ANTHROPIC_API_KEY を設定してください。")
    os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY

# --- Amazon Bedrock (Claude) ---
elif LLM_PROVIDER == "bedrock":
    AWS_ACCESS_KEY_ID = os.environ.get("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY = os.environ.get("AWS_SECRET_ACCESS_KEY")
    AWS_REGION_NAME = os.environ.get("AWS_REGION_NAME")

    if IS_COLAB: 
        if not AWS_ACCESS_KEY_ID: AWS_ACCESS_KEY_ID = userdata.get("AWS_ACCESS_KEY_ID")
        if not AWS_SECRET_ACCESS_KEY: AWS_SECRET_ACCESS_KEY = userdata.get("AWS_SECRET_ACCESS_KEY")
        if not AWS_REGION_NAME: AWS_REGION_NAME = userdata.get("AWS_REGION_NAME")

    if not AWS_REGION_NAME:
         raise ValueError("AWSリージョン名 (AWS_REGION_NAME) が設定されていません。Bedrock利用にはリージョン指定が必要です。")

    # 環境変数に設定 (boto3がこれらを自動で読み込む)
    if AWS_ACCESS_KEY_ID: os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
    if AWS_SECRET_ACCESS_KEY: os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
    os.environ["AWS_DEFAULT_REGION"] = AWS_REGION_NAME # boto3が参照する標準的なリージョン環境変数名
    os.environ["AWS_REGION"] = AWS_REGION_NAME # いくつかのライブラリはこちらを参照することもある

print(f"APIキー/環境変数の設定完了 (プロバイダー: {LLM_PROVIDER})")

### LLMクライアントの初期化

このセルは、上で選択・設定したLLMプロバイダーに基づいて、対応するLLMクライアントを初期化します。

In [ ]:
# === LLMクライアントの動的初期化 ===
llm = None

if LLM_PROVIDER == "openai":
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
elif LLM_PROVIDER == "azure":
    from langchain_openai import AzureChatOpenAI
    llm = AzureChatOpenAI(
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME"), # 環境変数から取得
        openai_api_version=os.environ.get("OPENAI_API_VERSION"), # 環境変数から取得
        temperature=0,
    )
elif LLM_PROVIDER == "google":
    from langchain_google_vertexai import ChatVertexAI
    # PROJECT_ID, LOCATION は前のセルで環境変数に設定済みか、Colabの場合は直接利用
    llm = ChatVertexAI(model_name="gemini-1.5-flash-001", temperature=0, project=os.environ.get("GOOGLE_CLOUD_PROJECT"), location=os.environ.get("GOOGLE_CLOUD_LOCATION"))
elif LLM_PROVIDER == "google_genai":
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0, convert_system_message_to_human=True)
elif LLM_PROVIDER == "anthropic":
    from langchain_anthropic import ChatAnthropic
    llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)
elif LLM_PROVIDER == "bedrock":
    from langchain_aws import ChatBedrock # langchain_community.chat_models から langchain_aws に変更の可能性あり
    # AWS_REGION_NAME は前のセルで環境変数 AWS_DEFAULT_REGION に設定済み
    llm = ChatBedrock( # BedrockChat ではなく ChatBedrock が一般的
        model_id="anthropic.claude-3-haiku-20240307-v1:0",
        # region_name=os.environ.get("AWS_DEFAULT_REGION"), # 通常、boto3が環境変数から自動で読み込む
        model_kwargs={"temperature": 0},
    )
else:
    raise ValueError(
        f"Unsupported LLM_PROVIDER: {LLM_PROVIDER}. "
        "Please choose from 'openai', 'azure', 'google', 'google_genai', 'anthropic', or 'bedrock'."
    )

print(f"LLM Provider: {LLM_PROVIDER}")
if llm:
    print(f"LLM Client Type: {type(llm)}")
    # モデル名取得の試行を汎用的に
    model_attr = getattr(llm, 'model', None) or \ 
                 getattr(llm, 'model_name', None) or \ 
                 getattr(llm, 'model_id', None) or \ 
                 (hasattr(llm, 'llm') and getattr(llm.llm, 'model', None)) # 一部のLLMクライアントのネスト構造に対応
    if hasattr(llm, 'azure_deployment') and not model_attr: # Azure特有の属性
        model_attr = llm.azure_deployment
        
    if model_attr:
        print(f"LLM Model: {model_attr}")
    else:
        print("LLM Model: (Could not determine model name from client attributes)")


---

この章では、スーパーバイザー型のタスク割り振りや複数エージェントによる対話シミュレーションなど、マルチエージェントシステムの構築方法を学びます。

まずは、演習全体で利用する基本的な状態定義と、エージェントノードやスーパーバイザーノードを簡単に作成するためのヘルパー関数を定義しましょう。

In [ ]:
from typing import TypedDict, Annotated, Optional, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import operator

# === 基本的な状態定義 ===
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    next_agent: Optional[str] # 次に実行するエージェントの名前
    task_description: Optional[str] # 現在のタスクの説明
    shared_data: Optional[dict] # エージェント間で共有するデータ

# === ヘルパー関数: エージェントノード作成 ===
def create_agent_node(llm_client, system_message_prompt: str):
    """特定の役割を持つエージェントノードを作成する関数"""
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_message_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])
    agent_runnable = prompt | llm_client
    def agent_node_func(state: AgentState):
        print(f"\n--- Executing Agent Node ({system_message_prompt[:30]}...) ---")
        print(f"Incoming messages: {state['messages'][-1].content if state['messages'] else 'No messages'}")
        response = agent_runnable.invoke(state) # messagesはstateから自動的に取得される
        print(f"Agent response: {response.content}")
        return {"messages": [response]} # 新しいメッセージをリストとして返す
    return agent_node_func

# === ヘルパー関数: スーパーバイザーノード作成 ===
def create_supervisor_node(llm_client, agent_names: list[str], system_prompt_template: str):
    """タスクを適切なエージェントにルーティングするスーパーバイザーノードを作成する関数"""
    # agent_names を "AgentA, AgentB, AgentC" のような文字列に変換
    agent_options_string = ", ".join(agent_names)
    
    prompt_template = system_prompt_template.format(
        agent_options=agent_options_string,
        # task_description プレースホルダーは外部から与えられる想定
        # messages プレースホルダーは MessagesPlaceholder で処理
    )
    
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=prompt_template),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "現在のタスクと会話履歴に基づいて、次に実行すべきエージェントを上記選択肢の中から一つだけ選んで、その名前のみを述べてください。追加のテキストは不要です。例: {example_agent_name}")
    ])
    supervisor_runnable = prompt | llm_client

    def supervisor_node_func(state: AgentState):
        print("\n--- Executing Supervisor Node ---")
        if not state.get("task_description") and state.get("messages"):
             # 最後のメッセージをタスク記述として使用（もしあれば）
            task_desc_from_message = state["messages"][-1].content
        elif state.get("task_description"):
            task_desc_from_message = state["task_description"]
        else:
            # 最初の実行でタスク説明がない場合、デフォルトの指示を出す
            # これは通常、グラフの入力で task_description を設定することで回避されるべき
            print("Warning: Supervisor called without explicit task_description or messages.")
            return {"next_agent": "END"} # または特定のエラー処理/デフォルトエージェント
        
        print(f"Supervisor: Current task/last message: {task_desc_from_message}")
        print(f"Supervisor: Routing based on agent options: {agent_options_string}")
        
        # runnableに渡すstateの内容を調整。MessagesPlaceholderが'messages'を期待するため。
        # また、プロンプト内でtask_descriptionを直接参照しない形式にしたため、ここでは不要。
        # HumanMessageの例として、最初のオプションを渡す
        invocation_payload = {
            "messages": state['messages'], 
            "example_agent_name": agent_names[0] if agent_names else "ANY_AGENT"
        }
        response = supervisor_runnable.invoke(invocation_payload)
        next_agent_name = response.content.strip()
        print(f"Supervisor decision: Route to '{next_agent_name}'")

        if next_agent_name in agent_names or next_agent_name == "END":
            return {"next_agent": next_agent_name}
        else:
            print(f"Warning: Supervisor chose an invalid agent '{next_agent_name}'. Defaulting to END.")
            return {"next_agent": "END"}
        
    return supervisor_node_func

print("Base AgentState and helper functions defined.")
# print(AgentState.model_json_schema()) # TypedDictなのでこれは使えない
import inspect
print("AgentState fields:", inspect.get_annotations(AgentState))


---

### ■ 問題001: 基本的な2エージェント会話

最もシンプルなマルチエージェントの形である、2つのエージェントによる対話を作成します。
ここでは、「リサーチャー」エージェントと「ライター」エージェントが、指定されたトピックについて数ターン対話します。
状態管理には `AgentState` を、エージェントノードの作成にはヘルパー関数 `create_agent_node` を使用します。
会話の進行は、どのエージェントが次に発言するかを決定するシンプルなルーター関数によって制御されます。

In [ ]:
# 解答欄001
from langgraph.graph import StateGraph, END

# --- エージェント定義 ---
researcher_system_prompt = "あなたはリサーチ専門家です。与えられたトピックについて関連情報やキーポイントを簡潔に提供してください。"
researcher_node = create_agent_node(llm, researcher_system_prompt)

writer_system_prompt = "あなたはライティング専門家です。リサーチャーから提供された情報に基づいて、読者向けの短い解説文を作成してください。"
writer_node = create_agent_node(llm, writer_system_prompt)

# --- ルーター関数 ---
MAX_TURNS = 2 # 各エージェントが1回ずつ発言する (リサーチャー -> ライター -> リサーチャー -> ライター -> END)

def two_agent_router(state: AgentState):
    messages = state['messages']
    current_turn = len(messages) -1 # 初期メッセージ(Human)があるので-1

    print(f"\n--- Two Agent Router ---")
    print(f"Current turn: {current_turn}, Max turns for sequence: {MAX_TURNS * 2}")
    print(f"Last message type: {type(messages[-1]).__name__ if messages else 'None'}")

    if current_turn >= MAX_TURNS * 2: # 全エージェントがMAX_TURNS回ずつ発言したら終了
        print("Routing to END")
        return END
    
    # 最初のメッセージがHumanMessageの場合、次はリサーチャー
    if isinstance(messages[-1], HumanMessage):
        print("Routing to researcher")
        return "researcher"
    # リサーチャーの次はライター
    elif messages[-1].name == researcher_node.__name__ or (hasattr(messages[-1], 'name') and messages[-1].name == getattr(researcher_node, '__name__', None)):
        # AIMessageにはデフォルトでname属性がない場合があるため、create_agent_node側で付与するか、型で判定
        # ここでは簡略化のため、直前のメッセージがAIMessageで、かつ次のターンがライターであると仮定
        # より堅牢にするには、create_agent_nodeでAIMessageにnameを付与する
        # 今回は system_prompt を使って仮の name のように扱う
        # と思ったが、AIMessageには .name がないので、 type(message[-1]) is AIMessage and state['next_agent'] == 'writer' のような判定が必要
        # もしくは、メッセージの送信元を state に持たせるのがよりクリーン
        # ここではシンプルに、Human -> researcher, AI -> writer (researcher), AI -> researcher (writer) のような交互を想定
        print("Routing to writer")
        return "writer"
    # ライターの次はリサーチャー
    else: # AIMessage from writer
        print("Routing to researcher")
        return "researcher"

# --- グラフ構築 ---
workflow = ____(AgentState)

workflow.add_node("researcher", researcher_node)
workflow.add_node("writer", writer_node)

# エントリポイントはユーザーからのメッセージなので、ルーターが最初に誰に渡すか決める
workflow.set_entry_point("____") # ルーターノード名（あとで定義）

# ルーターノードを追加
# workflow.add_node("router", two_agent_router) # これはダメ。Conditional Edgesのところで使う

workflow.add_conditional_edges(
    "____", # ルーターノード名（あとで定義）
    two_agent_router, # two_agent_router は callable であり、state を受け取り、次に遷移するノード名を返す
    {
        "researcher": "researcher",
        "writer": "writer",
        END: END
    }
)

# 各エージェントの処理が終わったら、再度ルーターに判断を仰ぐ
workflow.add_edge("researcher", "____") # ルーターノード名
workflow.add_edge("writer", "____")   # ルーターノード名

app = workflow.____()

# --- グラフの実行と結果表示 ---
topic = "LangGraphのマルチエージェント機能"
inputs = {"messages": [HumanMessage(content=f"{topic}について教えてください。")], "task_description": topic}

print(f"\n--- Running Graph for Topic: {topic} ---")
for s_chunk in app.stream(inputs, {"recursion_limit": 10}):
    print(f"\nOutput from node: {list(s_chunk.keys())[0]}")
    print(s_chunk)
    print("---")

final_state_q1 = app.invoke(inputs, {"recursion_limit": 10})
print("\n--- Final State Q1 ---")
for msg in final_state_q1['messages']:
    print(f"{type(msg).__name__} ({getattr(msg, 'name', 'N/A')}): {msg.content}")


<details><summary>解答001</summary>

``````python
# 解答001
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage # AIMessage をインポート

# --- エージェント定義 ---
# create_agent_node は前のセルで定義済みなので、ここでは呼び出すだけ
researcher_system_prompt = "あなたはリサーチ専門家です。与えられたトピックについて関連情報やキーポイントを簡潔に提供してください。"
researcher_node = create_agent_node(llm, researcher_system_prompt)

writer_system_prompt = "あなたはライティング専門家です。リサーチャーから提供された情報に基づいて、読者向けの短い解説文を作成してください。"
writer_node = create_agent_node(llm, writer_system_prompt)

# --- ルーター関数 ---
MAX_TURNS = 2 # 各エージェントがMAX_TURNS回ずつ応答する (リサーチャー -> ライター -> リサーチャー -> ライター -> END)

def two_agent_router(state: AgentState):
    messages = state['messages']
    # 初期メッセージ(HumanMessage)の後に続くAIメッセージの数をカウント
    # HumanMessageはカウントに含めないため、AIメッセージの数でターンを判断
    ai_message_count = sum(1 for msg in messages if isinstance(msg, AIMessage))

    print(f"\n--- Two Agent Router ---")
    print(f"AI message count: {ai_message_count}, Max AI messages for sequence: {MAX_TURNS * 2}")
    print(f"Last message type: {type(messages[-1]).__name__ if messages else 'None'}")

    if ai_message_count >= MAX_TURNS * 2:
        print("Routing to END")
        return END
    
    # 直前のメッセージがHumanMessageなら、次はリサーチャー
    if isinstance(messages[-1], HumanMessage):
        print("Routing to researcher")
        return "researcher"
    # 直前のメッセージがAIMessageの場合、次は相手のエージェント
    # ここでは、AIメッセージの数が奇数なら次はライター (リサーチャーが発言済み)
    # AIメッセージの数が偶数なら次はリサーチャー (ライターが発言済み)
    elif isinstance(messages[-1], AIMessage):
        if ai_message_count % 2 == 1: # リサーチャーが発言した (AIメッセージ1, 3, ...)
            print("Routing to writer")
            return "writer"
        else: # ライターが発言した (AIメッセージ2, 4, ...)
            print("Routing to researcher")
            return "researcher"
    else: # 予期しないメッセージタイプ
        print("Unexpected message type, routing to END")
        return END

# --- グラフ構築 ---
workflow = StateGraph(AgentState)

workflow.add_node("researcher", researcher_node)
workflow.add_node("writer", writer_node)

# エントリポイントはユーザーからのメッセージを受け、最初にルーターに渡す
# ルーターノードは実際には存在せず、set_entry_point の後、conditional_edges で指定する最初のノードが実質的な開始点
# そのため、最初に呼び出すべき分岐先ノードを持つ conditional_edges の起点ノード名を指定する
# ここでは、メッセージを受け取った後、常に two_agent_router で判断するので、
# 最初に two_agent_router を呼び出すための架空の開始点を設定し、そこから分岐する。
# LangGraphでは、set_entry_pointの後に conditional_edges を使う場合、そのconditional_edgesのsourceノードが実質的な開始点となることが多い。
# より明示的にするには、ダミーの開始ノードを置き、そこから conditional_edge で router_logic につなぐ。
# もしくは、最初の agent を entry_point にし、その agent の後に router_logic を置く。
# 今回は、最初に必ず router を呼びたいので、router を起点とする。

# ダミーノードや特定のエージェントをエントリポイントにするのではなく、
# どのエージェントから開始するかをルーター自身に判断させるため、
# グラフの開始直後にルーターロジックが評価されるようにします。
# そのため、add_conditional_edges のsource_node_nameをエントリポイントとします。
# ただし、add_conditional_edges はノードではないため、直接エントリポイントにできない。
# 解決策：ダミーの "entry_router_logic" ノード（何もしない）を作り、そこから分岐する。

def entry_router_logic_node(state: AgentState):
    print("\n--- Entry Router Logic Node (Pass-through) ---")
    # 何もせず、入力状態をそのまま返すことで、後続の conditional_edges が評価される
    return {}

workflow.add_node("entry_router", entry_router_logic_node)
workflow.set_entry_point("entry_router")

workflow.add_conditional_edges(
    "entry_router", # このノードの後に two_agent_router が評価される
    two_agent_router,
    {
        "researcher": "researcher",
        "writer": "writer",
        END: END
    }
)

# 各エージェントの処理が終わったら、再度ルーター(entry_router経由)に判断を仰ぐ
workflow.add_edge("researcher", "entry_router")
workflow.add_edge("writer", "entry_router")

app = workflow.compile()

# --- グラフの実行と結果表示 ---
topic = "LangGraphのマルチエージェント機能"
inputs = {"messages": [HumanMessage(content=f"{topic}について教えてください。")], "task_description": topic}

print(f"\n--- Running Graph for Topic: {topic} ---")
for s_chunk in app.stream(inputs, {"recursion_limit": 10}):
    print(f"\nOutput from node: {list(s_chunk.keys())[0]}")
    # messages の内容を表示
    current_messages = s_chunk.get(list(s_chunk.keys())[0]).get('messages')
    if current_messages:
        print(f"  {type(current_messages[-1]).__name__}: {current_messages[-1].content}")
    else:
        print(s_chunk) # messages がない場合はそのまま表示
    print("---")

final_state_q1 = app.invoke(inputs, {"recursion_limit": 10})
print("\n--- Final State Q1 ---")
for msg in final_state_q1['messages']:
    # AIMessage には name 属性がないため、typeで判定して表示を調整
    if isinstance(msg, HumanMessage):
        print(f"HumanMessage: {msg.content}")
    elif isinstance(msg, AIMessage):
        # どのエージェントからのメッセージかを示す情報は、このシンプルな構造では直接msgオブジェクトにはない
        # 解説で、より高度な方法（stateに送信者情報を含めるなど）に触れると良い
        print(f"AIMessage: {msg.content}")
    else:
        print(f"{type(msg).__name__}: {msg.content}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:**
    *   `create_agent_node` ヘルパー関数を使用して、異なる役割を持つ複数のエージェント（リサーチャー、ライター）を定義する方法。
    *   `AgentState` を使用して、エージェント間のメッセージ履歴を共有する方法。
    *   条件付きエッジ (`add_conditional_edges`) とルーター関数 (`two_agent_router`) を使用して、エージェント間の対話フローを制御する方法。
    *   固定されたターン数で対話を終了させる基本的な制御方法。
*   **コード解説:**
    *   **エージェント定義:** `create_agent_node` を使って `researcher_node` と `writer_node` を作成します。それぞれ異なるシステムプロンプトが与えられ、専門的な役割を担います。
    *   **ルーター関数 (`two_agent_router`):**
        *   現在の `AgentState` を受け取ります。
        *   `messages` リスト内の `AIMessage` の数をカウントし、これが事前に定義した `MAX_TURNS * 2`（各エージェントがMAX_TURNS回ずつ発言する総AIメッセージ数）に達したら `END` を返してグラフを終了させます。
        *   直前のメッセージが `HumanMessage` であれば、対話の開始なので `researcher` にルーティングします。
        *   直前のメッセージが `AIMessage` の場合、AIメッセージの総数（`ai_message_count`）が奇数ならリサーチャーが発言した後なので `writer` に、偶数ならライターが発言した後なので `researcher` にルーティングします。これにより、エージェントが交互に応答するようになります。
    *   **グラフ構築:**
        *   `StateGraph(AgentState)` でグラフを初期化します。
        *   `researcher` と `writer` ノードを追加します。
        *   `entry_router_logic_node` というシンプルなノードを追加し、これを `set_entry_point` でグラフの開始点として設定します。このノードは何も処理せず、状態をそのまま返すことで、直後に続く `add_conditional_edges` のルーターロジック (`two_agent_router`) が評価されるようにするためのものです。
        *   `add_conditional_edges` を使用して、`entry_router` ノードから `two_agent_router` 関数の結果に基づいて `researcher`、`writer`、または `END` に分岐させます。
        *   `researcher` ノードと `writer` ノードの実行後は、再び `entry_router` にエッジを接続し、次の行動をルーター関数に決定させます。
    *   **実行:**
        *   初期入力として、ユーザーからの質問 (`HumanMessage`) とタスクの説明を `AgentState` に設定します。
        *   `app.stream()` または `app.invoke()` でグラフを実行します。`recursion_limit` は、ループや条件分岐が多い場合に備えて設定しています。
    *   **AIMessageの送信元:** このシンプルな例では、`AIMessage` オブジェクト自体にはどのエージェントが生成したかの情報は含まれていません。ルーター関数内のロジック（例：AIメッセージの数）で間接的に判断しています。より複雑なシナリオでは、`AgentState` に `last_speaker` のようなフィールドを追加したり、`AIMessage` の `name` 属性（または `additional_kwargs`）に送信元エージェントの名前を明示的に含める方法が考えられます。

この演習により、複数のエージェントが協調して（この場合は交互に発言して）タスクを進める基本的なパターンをLangGraphでどのように構築できるかが理解できます。
---
</details>

---

### ■ 問題002: スーパーバイザーによるタスク割り振り

次に、スーパーバイザーエージェントを導入し、タスクの内容に応じて適切なワーカーエージェントに処理を割り振る方法を学びます。
ユーザーがタスクを要求すると、スーパーバイザーがその内容を判断し、「コード生成担当」「文章要約担当」「汎用チャット担当」のいずれかのエージェントにタスクをルーティングします。
ここでは、ヘルパー関数 `create_supervisor_node` と `create_agent_node` を活用します。

In [ ]:
# 解答欄002
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

# --- ワーカーエージェント定義 ---
code_generator_prompt = "あなたは優秀なプログラマーです。与えられた要件に基づいて、Pythonコードを生成してください。コードブロック（```python ... ```）で回答してください。"
code_generator_agent = create_agent_node(llm, code_generator_prompt)

text_summarizer_prompt = "あなたは文章要約の専門家です。与えられたテキストの重要なポイントを捉え、簡潔な要約を作成してください。"
text_summarizer_agent = create_agent_node(llm, text_summarizer_prompt)

general_chatbot_prompt = "あなたは親切なAIアシスタントです。ユーザーの質問に対して、丁寧かつ分かりやすく回答してください。"
general_chatbot_agent = create_agent_node(llm, general_chatbot_prompt)

worker_agent_names = ["CodeGenerator", "TextSummarizer", "GeneralChatbot"]

# --- スーパーバイザーエージェント定義 ---
supervisor_system_prompt_template = ( # .format(agent_options=..., task_description=...) を想定
    "あなたはタスク管理を行うスーパーバイザーです。"
    "ユーザーからのリクエスト（タスク記述）とこれまでの会話履歴を考慮し、以下の専門エージェントの中から最も適切と思われるものを一つだけ選んでください。"
    "選択肢: {agent_options}"
    "ユーザーのリクエストは次の通りです: 「{{task_description}}」"
    "もしどのエージェントも適切でないと判断した場合は、'END'と答えてください。"
)

# supervisor_node の作成 (create_supervisor_node はプロンプト内で task_description を直接使わない想定なので調整)
# create_supervisor_node は、渡された system_prompt_template に agent_options を format し、
# その後、固定の user メッセージを追加してプロンプトを完成させます。
# task_description は state['task_description'] または state['messages'][-1].content から取得される。
supervisor_prompt_for_helper = (
    "あなたはタスク管理を行うスーパーバイザーです。"
    "ユーザーからのリクエスト（直近のメッセージやタスク記述で示される）とこれまでの会話履歴を考慮し、"
    "以下の専門エージェントの中から最も適切と思われるものを一つだけ選んでください。"
    "選択肢: {agent_options}"
    "もしどのエージェントも適切でないと判断した場合は、'END'と答えてください。"
)

supervisor_agent = create_supervisor_node(llm, worker_agent_names, supervisor_prompt_for_helper)

# --- ルーター関数 (スーパーバイザーの決定に基づく) ---
def supervisor_router(state: AgentState):
    print(f"\n--- Supervisor Router ---")
    next_agent = state.get("next_agent")
    print(f"Next agent decided by supervisor: {next_agent}")
    if next_agent in worker_agent_names:
        return next_agent
    elif next_agent == "END":
        return END
    else: # 念のため
        print(f"Warning: Unknown next_agent '{next_agent}'. Routing to END.")
        return END

# --- グラフ構築 ---
workflow_q2 = ____(AgentState) # StateGraph

workflow_q2.add_node("Supervisor", supervisor_agent)
workflow_q2.add_node("CodeGenerator", code_generator_agent)
workflow_q2.add_node("TextSummarizer", text_summarizer_agent)
workflow_q2.add_node("GeneralChatbot", general_chatbot_agent)

workflow_q2.set_entry_point("____") # Supervisor

workflow_q2.add_conditional_edges(
    "Supervisor",
    ____, # supervisor_router
    {
        "CodeGenerator": "CodeGenerator",
        "TextSummarizer": "TextSummarizer",
        "GeneralChatbot": "GeneralChatbot",
        END: END
    }
)

# ワーカーエージェントの処理が終わったらグラフを終了
workflow_q2.add_edge("CodeGenerator", ____) # END
workflow_q2.add_edge("TextSummarizer", ____) # END
workflow_q2.add_edge("GeneralChatbot", ____) # END

app_q2 = workflow_q2.____() # compile

# --- グラフの実行と結果表示 ---
tasks = [
    {"task_description": "Pythonでフィボナッチ数列を計算する関数を書いてください。", "messages": [HumanMessage(content="Pythonでフィボナッチ数列を計算する関数を書いてください。")]},
    {"task_description": "LangGraphは、複雑なLLMアプリケーションを構築するためのライブラリです。状態を持つグラフを定義し、ノードとエッジで処理フローを表現できます。マルチエージェントシステムや自己修正ループなどを実装するのに役立ちます。この説明を3行で要約してください。", 
     "messages": [HumanMessage(content="LangGraphは、複雑なLLMアプリケーションを構築するためのライブラリです。状態を持つグラフを定義し、ノードとエッジで処理フローを表現できます。マルチエージェントシステムや自己修正ループなどを実装するのに役立ちます。この説明を3行で要約してください。")]},
    {"task_description": "明日の天気は？", "messages": [HumanMessage(content="明日の天気は？")]},
    {"task_description": "ありがとう、助かったよ！", "messages": [HumanMessage(content="ありがとう、助かったよ！")]} # ENDになるはず
]

for i, task_input in enumerate(tasks):
    print(f"\n--- Running Graph for Task Q2-{i+1}: {task_input['task_description'][:50]}... ---")
    # ストリーミングで途中経過を表示
    for s_chunk in app_q2.stream(task_input, {"recursion_limit": 5}):
        node_name = list(s_chunk.keys())[0]
        print(f"\nOutput from node: {node_name}")
        content = s_chunk[node_name]
        # messages があれば最新のものを表示、なければそのまま表示
        if isinstance(content, dict) and 'messages' in content and content['messages']:
            latest_message = content['messages'][-1]
            print(f"  {type(latest_message).__name__}: {latest_message.content}")
        elif isinstance(content, dict) and 'next_agent' in content:
             print(f"  Next agent: {content['next_agent']}")
        else:
            print(f"  Content: {content}")
        print("---")
    
    final_state = app_q2.invoke(task_input, {"recursion_limit": 5})
    print(f"\n--- Final State Q2-{i+1} ---")
    if final_state.get('messages'):
        # 最後のAIメッセージ（エージェントの応答）を表示
        final_ai_message = next((msg for msg in reversed(final_state['messages']) if isinstance(msg, AIMessage)), None)
        if final_ai_message:
            print(f"Final AI Message: {final_ai_message.content}")
        else:
            print(f"No final AI message. Last message: {final_state['messages'][-1].content if final_state['messages'] else 'None'}")
    else:
        print("No messages in final state.")
    print(f"Routed to: {final_state.get('next_agent')}") # Supervisorが最後に決定したnext_agent (ENDのはず)


<details><summary>解答002</summary>

``````python
# 解答002
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage # AIMessage をインポート

# --- ワーカーエージェント定義 ---
code_generator_prompt = "あなたは優秀なプログラマーです。与えられた要件に基づいて、Pythonコードを生成してください。コードブロック（```python ... ```）で回答してください。"
code_generator_agent = create_agent_node(llm, code_generator_prompt)

text_summarizer_prompt = "あなたは文章要約の専門家です。与えられたテキストの重要なポイントを捉え、簡潔な要約を作成してください。"
text_summarizer_agent = create_agent_node(llm, text_summarizer_prompt)

general_chatbot_prompt = "あなたは親切なAIアシスタントです。ユーザーの質問に対して、丁寧かつ分かりやすく回答してください。"
general_chatbot_agent = create_agent_node(llm, general_chatbot_prompt)

worker_agent_names = ["CodeGenerator", "TextSummarizer", "GeneralChatbot"]

# --- スーパーバイザーエージェント定義 ---
# supervisor_agent のプロンプトは create_supervisor_node ヘルパー関数内で適切に組まれる
supervisor_prompt_for_helper = (
    "あなたはタスク管理を行うスーパーバイザーです。"
    "ユーザーからのリクエスト（直近のメッセージやタスク記述で示される）とこれまでの会話履歴を考慮し、"
    "以下の専門エージェントの中から最も適切と思われるものを一つだけ選んでください。"
    "選択肢: {agent_options}"
    "もしどのエージェントも適切でない、またはタスクが完了したと思われる場合は、'END'と答えてください。"
)
supervisor_agent = create_supervisor_node(llm, worker_agent_names, supervisor_prompt_for_helper)

# --- ルーター関数 (スーパーバイザーの決定に基づく) ---
def supervisor_router(state: AgentState):
    print(f"\n--- Supervisor Router ---")
    next_agent = state.get("next_agent")
    print(f"Next agent decided by supervisor: {next_agent}")
    if next_agent in worker_agent_names:
        return next_agent
    elif next_agent == "END":
        return END
    else: 
        print(f"Warning: Unknown next_agent '{next_agent}' from supervisor. Routing to END.")
        return END

# --- グラフ構築 ---
workflow_q2 = StateGraph(AgentState)

workflow_q2.add_node("Supervisor", supervisor_agent)
workflow_q2.add_node("CodeGenerator", code_generator_agent)
workflow_q2.add_node("TextSummarizer", text_summarizer_agent)
workflow_q2.add_node("GeneralChatbot", general_chatbot_agent)

workflow_q2.set_entry_point("Supervisor")

workflow_q2.add_conditional_edges(
    "Supervisor",
    supervisor_router, 
    {
        "CodeGenerator": "CodeGenerator",
        "TextSummarizer": "TextSummarizer",
        "GeneralChatbot": "GeneralChatbot",
        END: END
    }
)

# ワーカーエージェントの処理が終わったらグラフを終了
workflow_q2.add_edge("CodeGenerator", END)
workflow_q2.add_edge("TextSummarizer", END)
workflow_q2.add_edge("GeneralChatbot", END)

app_q2 = workflow_q2.compile()

# --- グラフの実行と結果表示 (変更なし) ---
tasks = [
    {"task_description": "Pythonでフィボナッチ数列を計算する関数を書いてください。", "messages": [HumanMessage(content="Pythonでフィボナッチ数列を計算する関数を書いてください。")]},
    {"task_description": "LangGraphは、複雑なLLMアプリケーションを構築するためのライブラリです。状態を持つグラフを定義し、ノードとエッジで処理フローを表現できます。マルチエージェントシステムや自己修正ループなどを実装するのに役立ちます。この説明を3行で要約してください。", 
     "messages": [HumanMessage(content="LangGraphは、複雑なLLMアプリケーションを構築するためのライブラリです。状態を持つグラフを定義し、ノードとエッジで処理フローを表現できます。マルチエージェントシステムや自己修正ループなどを実装するのに役立ちます。この説明を3行で要約してください。")]},
    {"task_description": "明日の天気は？", "messages": [HumanMessage(content="明日の天気は？")]},
    {"task_description": "ありがとう、助かったよ！", "messages": [HumanMessage(content="ありがとう、助かったよ！")]} # ENDになるはず
]

for i, task_input in enumerate(tasks):
    print(f"\n--- Running Graph for Task Q2-{i+1}: {task_input['task_description'][:50]}... ---")
    for s_chunk in app_q2.stream(task_input, {"recursion_limit": 5}):
        node_name = list(s_chunk.keys())[0]
        print(f"\nOutput from node: {node_name}")
        content = s_chunk[node_name]
        if isinstance(content, dict) and 'messages' in content and content['messages']:
            latest_message = content['messages'][-1]
            print(f"  {type(latest_message).__name__}: {latest_message.content}")
        elif isinstance(content, dict) and 'next_agent' in content:
             print(f"  Next agent: {content['next_agent']}")
        else:
            print(f"  Content: {content}")
        print("---")
    
    final_state = app_q2.invoke(task_input, {"recursion_limit": 5})
    print(f"\n--- Final State Q2-{i+1} ---")
    if final_state.get('messages'):
        final_ai_message = next((msg for msg in reversed(final_state['messages']) if isinstance(msg, AIMessage)), None)
        if final_ai_message:
            print(f"Final AI Message: {final_ai_message.content}")
        elif final_state['messages']:
             print(f"Last message (not AI): {final_state['messages'][-1].content}")
        else:
            print("No messages in final state.")
    else:
        print("No messages in final state.")
    # Supervisorが最後に決定したnext_agent (ENDのはずだが、ワーカー実行後は supervisor router を通らないので supervisor の決定が最終となる)
    # Supervisor の出力が next_agent なのでそれを表示
    if 'Supervisor' in final_state:
        print(f"Supervisor's final decision for 'next_agent': {final_state['Supervisor'].get('next_agent')}")
    elif final_state.get('next_agent'): # Supervisor を通らなかった場合 (エラーなど)
        print(f"Final 'next_agent' in state: {final_state.get('next_agent')}")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:**
    *   `create_supervisor_node` ヘルパー関数を使用して、タスクを複数のワーカーエージェントに割り振るスーパーバイザーを定義する方法。
    *   `create_agent_node` を用いて、それぞれ専門的な役割を持つワーカーエージェント（コード生成、文章要約、汎用チャット）を作成する方法。
    *   スーパーバイザーの決定 (`state['next_agent']`) に基づいて、`add_conditional_edges` を使って適切なワーカーエージェントに処理を分岐させる方法。
    *   ワーカーエージェントの処理完了後、グラフを `END` で終了させる単純な一方向のフロー。
*   **コード解説:**
    *   **ワーカーエージェント定義:** `CodeGenerator`、`TextSummarizer`、`GeneralChatbot` という3つのワーカーエージェントを `create_agent_node` を使って作成します。それぞれのシステムプロンプトで役割を特化させています。
    *   **スーパーバイザーエージェント定義:**
        *   `supervisor_prompt_for_helper` は、スーパーバイザーの基本的な指示を定義します。`{agent_options}` はヘルパー関数によって実際のワーカーエージェント名リストに置き換えられます。このプロンプトは、ユーザーのタスク記述（直近のメッセージや `state['task_description']` から取得）と会話履歴を基に、次に実行すべきエージェント名（または `END`）を出力するようLLMに指示します。
        *   `create_supervisor_node(llm, worker_agent_names, supervisor_prompt_for_helper)` でスーパーバイザーノード (`supervisor_agent`) を作成します。このヘルパー関数は、渡されたプロンプトとエージェント名リストを使って、LLMが次のエージェントを選択するロジックを内包しています。
    *   **ルーター関数 (`supervisor_router`):**
        *   この関数は、スーパーバイザーノードが `state['next_agent']` に設定した値に基づいて、次にどのノードを実行するかを決定します。
        *   `state['next_agent']` がワーカーエージェント名のいずれかと一致すればそのエージェント名を、`"END"` であれば `END` を返します。
    *   **グラフ構築:**
        *   `StateGraph(AgentState)` でグラフを初期化します。
        *   スーパーバイザーノードと3つのワーカーエージェントノードを追加します。
        *   `set_entry_point("Supervisor")` で、グラフの開始点をスーパーバイザーに設定します。
        *   `add_conditional_edges` を使用して、`Supervisor` ノードから `supervisor_router` 関数の結果に基づいて、各ワーカーエージェントまたは `END` に分岐させます。
        *   各ワーカーエージェント (`CodeGenerator`, `TextSummarizer`, `GeneralChatbot`) の処理が完了したら、`add_edge` を使って直接 `END` に接続し、グラフを終了させます。
    *   **実行:**
        *   複数の異なるタスク (`tasks` リスト) を用意し、それぞれについてグラフを実行します。
        *   入力 (`task_input`) には `task_description` と初期 `messages` を含めます。スーパーバイザーは主に `task_description`（または最新のメッセージ）を見て判断します。
        *   `app_q2.stream()` で実行中の各ノードの出力（メッセージや次のエージェントの決定）を、`app_q2.invoke()` で最終状態を確認します。

この演習では、中央のスーパーバイザーがタスクを解析し、適切な専門家（ワーカーエージェント）に処理を委任するという、より実践的なマルチエージェントのパターンを構築します。ヘルパー関数を利用することで、各コンポーネントの定義が簡潔になっている点にも注目してください。
---
</details>

---

### ■ 問題003: スーパーバイザーによる複数エージェントの逐次連携

この演習では、スーパーバイザーが複数のエージェントの作業を順番に連携させる、より複雑なワークフローを構築します。
例えば、「トピックXについてリサーチし、その結果を基に記事を作成する」というタスクを考えます。
スーパーバイザーはまず「リサーチャー」エージェントに指示を出し、リサーチャーが完了したら次に「ライター」エージェントに指示を出す、といった流れを制御します。
スーパーバイザーは、各ステップの完了後に次にどのアクション（別のエージェントの呼び出し、ユーザーへの確認、終了など）を取るべきか判断します。

In [ ]:
# 解答欄003
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage

# --- エージェント定義 (問題001から再利用または微調整) ---
research_agent_prompt_q3 = "あなたはリサーチ専門家です。与えられたトピックまたは質問について、関連性の高い情報を収集し、キーポイントをまとめて報告してください。"
research_agent_q3 = create_agent_node(llm, research_agent_prompt_q3)

writer_agent_prompt_q3 = "あなたはプロのライターです。提供された情報やキーポイントに基づいて、指示された形式（例：ブログ記事、要約、メール文面など）で文章を作成してください。"
writer_agent_q3 = create_agent_node(llm, writer_agent_prompt_q3)

sequential_worker_names = ["Researcher", "Writer"] # スーパーバイザーが選択可能なワーカー

# --- スーパーバイザーエージェント定義 (逐次連携用) ---
supervisor_prompt_sequential = (
    "あなたは高度なタスク管理を行うAIスーパーバイザーです。"
    "ユーザーからの初期リクエストと、これまでのエージェントの作業履歴（メッセージ）に基づいて、次に実行すべきアクションを決定します。"
    "選択可能なアクションは以下の通りです: {agent_options}。"
    "タスクの初期リクエスト: 「{{task_description}}」"
    "現在の会話/作業履歴:
    {{#each messages}}{{this.type}}: {{this.content}}\n{{/each}}"
    "考慮事項:
    - ユーザーの最終的な目標は何か？
    - 現在の履歴で、目標達成に必要な情報は揃っているか？
    - 次にResearcherを呼ぶべきか？（例: 情報が不足している、新しい観点が必要）
    - 次にWriterを呼ぶべきか？（例: リサーチ結果を基に文章作成が必要）
    - 全ての作業が完了したと思われるか？その場合は 'END' と答えてください。"
    "指示: 上記を考慮し、次に実行すべきアクション（{agent_options} または END）の名前のみを述べてください。"
)
# create_supervisor_node は MessagesPlaceholder を使うので、プロンプトテンプレートの messages 部分は実際には不要
supervisor_prompt_for_helper_q3 = (
    "あなたは高度なタスク管理を行うAIスーパーバイザーです。"
    "ユーザーからの初期リクエスト（タスク記述で示される）と、これまでのエージェントの作業履歴（メッセージ）に基づいて、次に実行すべきアクションを決定します。"
    "選択可能なアクションは以下の通りです: {agent_options}。"
    "ユーザーの初期リクエストはタスク記述や最初のメッセージで確認できます。"
    "現在の会話/作業履歴をよく読んでください。"
    "考慮事項:
    - ユーザーの最終的な目標は何か？
    - 現在の履歴で、目標達成に必要な情報は揃っているか？
    - 次にResearcherを呼ぶべきか？（例: 情報が不足している、新しい観点が必要）
    - 次にWriterを呼ぶべきか？（例: リサーチ結果を基に文章作成が必要）
    - 全ての作業が完了したと思われるか？その場合は 'END' と答えてください。"
    "指示: 上記を考慮し、次に実行すべきアクション（{agent_options} または END）の名前のみを述べてください。"
)

supervisor_agent_q3 = create_supervisor_node(llm, sequential_worker_names, supervisor_prompt_for_helper_q3)

# --- ルーター関数 (スーパーバイザーの決定に基づく) ---
# 問題002のsupervisor_routerを再利用可能だが、明確化のため別名で定義してもよい
def sequential_supervisor_router(state: AgentState):
    print(f"\n--- Sequential Supervisor Router ---")
    next_agent = state.get("next_agent")
    sender = state.get("messages", [])[-1].type if state.get("messages") else "unknown" # 直前の発言者タイプ
    print(f"Decision by supervisor (after {sender}'s turn): Route to '{next_agent}'")
    
    if next_agent in sequential_worker_names:
        return next_agent
    elif next_agent == "END":
        return END
    else:
        print(f"Warning: Unknown next_agent '{next_agent}' from supervisor. Defaulting to END.")
        return END

# --- グラフ構築 ---
workflow_q3 = ____(AgentState) # StateGraph

workflow_q3.add_node("Supervisor", supervisor_agent_q3)
workflow_q3.add_node("Researcher", research_agent_q3)
workflow_q3.add_node("Writer", writer_agent_q3)

workflow_q3.set_entry_point("____") # Supervisor

workflow_q3.add_conditional_edges(
    "Supervisor",
    ____, # sequential_supervisor_router
    {
        "Researcher": "Researcher",
        "Writer": "Writer",
        END: END
    }
)

# ワーカーエージェントの処理が終わったら、再度スーパーバイザーに判断を仰ぐ
workflow_q3.add_edge("Researcher", "____") # Supervisor
workflow_q3.add_edge("Writer", "____")     # Supervisor

app_q3 = workflow_q3.____() # compile

# --- グラフの実行と結果表示 ---
task_q3_description = "LangGraphのConditional Edges機能についてリサーチし、その主な利点を説明する短いブログ記事を作成してください。"
inputs_q3 = {
    "messages": [HumanMessage(content=task_q3_description)],
    "task_description": task_q3_description
}

print(f"\n--- Running Graph for Sequential Task Q3: {task_q3_description[:60]}... ---")
for s_chunk in app_q3.stream(inputs_q3, {"recursion_limit": 10}): # recursion_limit を少し増やす
    node_name = list(s_chunk.keys())[0]
    print(f"\nOutput from node: {node_name}")
    content = s_chunk[node_name]
    if isinstance(content, dict) and 'messages' in content and content['messages']:
        latest_message = content['messages'][-1]
        print(f"  {type(latest_message).__name__} ({getattr(latest_message, 'name', '')}): {latest_message.content}")
    elif isinstance(content, dict) and 'next_agent' in content:
         print(f"  Next agent: {content['next_agent']}")
    else:
        print(f"  Content: {content}")
    print("---")

final_state_q3 = app_q3.invoke(inputs_q3, {"recursion_limit": 10})
print("\n--- Final State Q3 ---")
for msg in final_state_q3['messages']:
    msg_type = type(msg).__name__
    speaker = getattr(msg, 'name', 'N/A') if isinstance(msg, AIMessage) else msg_type
    # AIMessage の name 属性は通常ないので、create_agent_node で付与するか、ここでは固定の文字列で示す
    # supervisorからのメッセージはnameがないので、contentを見るしかない
    if msg_type == "AIMessage":
        # 簡単のため、Supervisorからの指示メッセージとエージェントの応答を区別
        if any(agent_name in msg.content for agent_name in sequential_worker_names + ["END"] ) and len(msg.content) < 20:
            speaker = "SupervisorDecision"
        else:
            # どのエージェントからのメッセージか特定するのは難しい。メッセージ内容から推測するか、stateに情報が必要。
            # ここでは単純にAIMessageとしておく
            speaker = "AgentResponse"
    print(f"{speaker}: {msg.content}")


<details><summary>解答003</summary>

``````python
# 解答003
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage

# --- エージェント定義 (問題001から再利用または微調整) ---
research_agent_prompt_q3 = "あなたはリサーチ専門家です。与えられたトピックまたは質問について、関連性の高い情報を収集し、キーポイントをまとめて報告してください。"
research_agent_q3 = create_agent_node(llm, research_agent_prompt_q3)

writer_agent_prompt_q3 = "あなたはプロのライターです。提供された情報やキーポイントに基づいて、指示された形式（例：ブログ記事、要約、メール文面など）で文章を作成してください。"
writer_agent_q3 = create_agent_node(llm, writer_agent_prompt_q3)

sequential_worker_names = ["Researcher", "Writer"] # スーパーバイザーが選択可能なワーカー

# --- スーパーバイザーエージェント定義 (逐次連携用) ---
supervisor_prompt_for_helper_q3 = (
    "あなたは高度なタスク管理を行うAIスーパーバイザーです。"
    "ユーザーからの初期リクエスト（タスク記述で示される）と、これまでのエージェントの作業履歴（メッセージ）に基づいて、次に実行すべきアクションを決定します。"
    "選択可能なアクションは以下の通りです: {agent_options}。"
    "ユーザーの初期リクエストはタスク記述や最初のメッセージで確認できます。"
    "現在の会話/作業履歴をよく読んでください。"
    "考慮事項:\n"
    "- ユーザーの最終的な目標は何か？\n"
    "- 現在の履歴で、目標達成に必要な情報は揃っているか？\n"
    "- 次にResearcherを呼ぶべきか？（例: 情報が不足している、新しい観点が必要）\n"
    "- 次にWriterを呼ぶべきか？（例: リサーチ結果を基に文章作成が必要）\n"
    "- 全ての作業が完了したと思われるか？その場合は 'END' と答えてください。"
    "指示: 上記を考慮し、次に実行すべきアクション（{agent_options} または END）の名前のみを述べてください。"
)
supervisor_agent_q3 = create_supervisor_node(llm, sequential_worker_names, supervisor_prompt_for_helper_q3)

# --- ルーター関数 (スーパーバイザーの決定に基づく) ---
def sequential_supervisor_router(state: AgentState):
    print(f"\n--- Sequential Supervisor Router ---")
    next_agent = state.get("next_agent")
    # messages が存在し、かつ空でないことを確認
    sender_indicator = "unknown"
    if state.get("messages") and state["messages"]:
        last_message = state["messages"][-1]
        if isinstance(last_message, HumanMessage):
            sender_indicator = "Human"
        elif isinstance(last_message, AIMessage):
            # AIMessage の場合、それがSupervisorからの決定か、Agentの作業結果かを判断するのは難しい
            # ここでは単純に "AI" とする。より詳細な判別は state や message に追加情報が必要。
            sender_indicator = "AI (Supervisor/Agent)"
    print(f"Decision by supervisor (after {sender_indicator}'s turn): Route to '{next_agent}'")
    
    if next_agent in sequential_worker_names:
        return next_agent
    elif next_agent == "END":
        return END
    else:
        print(f"Warning: Unknown next_agent '{next_agent}' from supervisor. Defaulting to END.")
        return END

# --- グラフ構築 ---
workflow_q3 = StateGraph(AgentState)

workflow_q3.add_node("Supervisor", supervisor_agent_q3)
workflow_q3.add_node("Researcher", research_agent_q3)
workflow_q3.add_node("Writer", writer_agent_q3)

workflow_q3.set_entry_point("Supervisor")

workflow_q3.add_conditional_edges(
    "Supervisor",
    sequential_supervisor_router,
    {
        "Researcher": "Researcher",
        "Writer": "Writer",
        END: END
    }
)

# ワーカーエージェントの処理が終わったら、再度スーパーバイザーに判断を仰ぐ
workflow_q3.add_edge("Researcher", "Supervisor")
workflow_q3.add_edge("Writer", "Supervisor")

app_q3 = workflow_q3.compile()

# --- グラフの実行と結果表示 (変更なし) ---
task_q3_description = "LangGraphのConditional Edges機能についてリサーチし、その主な利点を説明する短いブログ記事を作成してください。"
inputs_q3 = {
    "messages": [HumanMessage(content=task_q3_description)],
    "task_description": task_q3_description
}

print(f"\n--- Running Graph for Sequential Task Q3: {task_q3_description[:60]}... ---")
for s_chunk in app_q3.stream(inputs_q3, {"recursion_limit": 10}):
    node_name = list(s_chunk.keys())[0]
    print(f"\nOutput from node: {node_name}")
    content = s_chunk[node_name]
    if isinstance(content, dict) and 'messages' in content and content['messages']:
        latest_message = content['messages'][-1]
        print(f"  {type(latest_message).__name__}: {latest_message.content}")
    elif isinstance(content, dict) and 'next_agent' in content:
         print(f"  Next agent: {content['next_agent']}")
    else:
        print(f"  Content: {content}")
    print("---")

final_state_q3 = app_q3.invoke(inputs_q3, {"recursion_limit": 10})
print("\n--- Final State Q3 ---")
for msg_idx, msg in enumerate(final_state_q3['messages']):
    msg_type = type(msg).__name__
    speaker_info = msg_type # Default speaker info

    if isinstance(msg, HumanMessage):
        speaker_info = "User"
    elif isinstance(msg, AIMessage):
        # Try to infer if it's a supervisor decision or agent work
        # This is a heuristic. A more robust way is to add 'name' to AIMessage in create_agent_node
        # or have supervisor add a specifically named message.
        is_supervisor_decision = False
        if msg_idx > 0: # Check previous message to see if it was from Supervisor node
            # This logic is tricky because the supervisor's decision (next_agent) isn't a message itself
            # but rather an update to the state. The AIMessage is from the LLM *inside* the supervisor.
            # A simple check: if content is short and one of the agent names or END.
            if msg.content.strip() in sequential_worker_names + ["END"] and len(msg.content.strip()) < 20:
                 is_supervisor_decision = True 
        
        if is_supervisor_decision:
            speaker_info = "Supervisor (Decision)"
        else:
            # If not a clear supervisor decision, assume it's from a worker or supervisor's reasoning.
            # To differentiate worker, we'd need to know which worker ran before this message.
            # For simplicity, we'll label it broadly.
            # We can check the node that produced this message if we had that info here.
            speaker_info = "AI (Agent/Supervisor Reasoning)"
            
    print(f"[{msg_idx}] {speaker_info}: {msg.content}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:**
    *   スーパーバイザーが複数のエージェント（リサーチャー、ライター）の作業を逐次的に（順番に）連携させる方法。
    *   ワーカーエージェントの処理完了後、再びスーパーバイザーに制御を戻し、次のステップを判断させるループ構造の構築。
    *   スーパーバイザーのプロンプトを工夫し、現在のタスクの進行状況（メッセージ履歴）を考慮して次の行動（別のエージェントの呼び出し、または終了）を決定できるようにする方法。
    *   `task_description` を状態 (`AgentState`) に含めることで、スーパーバイザーが長期的なタスク目標を記憶しやすくする方法。
*   **コード解説:**
    *   **エージェント定義:** `research_agent_q3` と `writer_agent_q3` を `create_agent_node` を使って定義します。それぞれの役割は明確です。
    *   **スーパーバイザーエージェント定義 (`supervisor_agent_q3`):**
        *   `supervisor_prompt_for_helper_q3` は、この逐次連携シナリオに合わせて調整されています。プロンプト内で、現在のタスク記述 (`task_description`) と会話履歴 (`messages`) を考慮し、次に `Researcher` を呼ぶべきか、`Writer` を呼ぶべきか、あるいはタスク完了として `END` とすべきかを判断するよう指示しています。
        *   `create_supervisor_node` ヘルパー関数を使用してスーパーバイザーノードを作成します。このノードはLLMを呼び出し、その結果（次に実行すべきエージェント名または `END`）を `state['next_agent']` に設定します。
    *   **ルーター関数 (`sequential_supervisor_router`):**
        *   問題002のルーターとほぼ同じで、`state['next_agent']` の値に基づいて次に実行するノード名を返します。
    *   **グラフ構築:**
        *   `Supervisor`、`Researcher`、`Writer` の各ノードをグラフに追加します。
        *   `set_entry_point("Supervisor")` で、最初にスーパーバイザーがタスクを受け取ります。
        *   `add_conditional_edges` を使い、`Supervisor` ノードの後は `sequential_supervisor_router` の判断に基づいて `Researcher`、`Writer`、または `END` に分岐します。
        *   **重要な変更点:** `Researcher` ノードと `Writer` ノードの処理が完了した後、エッジは `END` ではなく、再び `"Supervisor"` に接続されます (`workflow_q3.add_edge("Researcher", "Supervisor")`)。これにより、ワーカーエージェントの作業結果をスーパーバイザーが確認し、次のステップ（例: Writerの呼び出し、または終了）を判断できるようになります。このループ構造が逐次連携の鍵です。
    *   **実行:**
        *   タスクとして「LangGraphのConditional Edges機能についてリサーチし、ブログ記事を作成する」という複合的な指示を与えます。
        *   スーパーバイザーはまずリサーチが必要と判断し `Researcher` を呼び出すでしょう。リサーチ結果がメッセージ履歴に追加された後、制御はスーパーバイザーに戻ります。スーパーバイザーはリサーチ結果を見て、次に記事作成のために `Writer` を呼び出すはずです。ライターが記事を完成させると、再び制御はスーパーバイザーに戻り、今度はタスク完了として `END` を選択することが期待されます。
        *   `recursion_limit` は、このようなループ構造を持つグラフで、意図しない無限ループを防ぐために重要です。

この演習は、スーパーバイザーを中心としたより洗練されたマルチエージェントワークフローを示しています。エージェントが単発のタスクをこなすだけでなく、スーパーバイザーの指示のもとで複数のステップを経て共同でより大きな目標を達成する様子をシミュレートしています。
---
</details>

---

### ■ 問題004: 階層型エージェント構造（簡易版）

この演習では、階層的なエージェント構造の基本的な概念をシミュレートします。
主担当エージェント（例：「旅行プランナー」）がタスクを実行中に、特定のサブタスク（例：「レビューを英語に翻訳する」）の処理が必要になったと判断した場合、そのサブタスクを専門のサブエージェント（例：「翻訳エージェント」）に依頼するような流れを作ります。

具体的には、以下の流れを実装します：
1. ユーザーが旅行プランナーに外国語のレビューを含む旅行計画の相談をします。
2. 旅行プランナーはレビューの翻訳が必要だと判断し、スーパーバイザーに「翻訳タスク」を実行するよう要求（特定のメッセージ形式で伝える）。
3. スーパーバイザーは「翻訳タスク」を翻訳エージェントに割り当てます。
4. 翻訳エージェントが翻訳を実行し、結果を返します。
5. スーパーバイザーは翻訳結果を受け取り、次に旅行プランナーに処理を戻します。
6. 旅行プランナーは翻訳されたレビューを元に、計画を続行します。
7. 最終的に旅行プランナーが計画を完成させたら、スーパーバイザーがENDを判断します。

この演習では、`AgentState` に `sub_task_result` のようなフィールドを追加して、サブエージェントの結果を主担当エージェントに渡すことを検討します。
スーパーバイザーのプロンプトも、このようなサブタスクの委任と結果の統合を考慮するように調整が必要です。

In [ ]:
# 解答欄004
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage
from typing import Sequence, TypedDict, Annotated, Optional
import operator

# --- 状態定義の更新 (サブタスク結果と元のタスク情報を保持) ---
class HierarchicalAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    next_agent: Optional[str]
    task_description: Optional[str]
    original_requester: Optional[str] # サブタスク完了後に戻るべきエージェント
    sub_task_description: Optional[str] # サブタスクの内容
    sub_task_result: Optional[str] # サブタスクの実行結果
    # shared_data: Optional[dict] # AgentStateから継承されるが、ここでは明示しない

# --- エージェント定義 ---
travel_planner_prompt = (
    "あなたは旅行プランナーです。ユーザーの要望に基づいて旅行プランを作成します。"
    "必要に応じて、外国語のレビューを翻訳するために「TRANSLATE:[レビューテキスト]」という形式で翻訳リクエストを生成できます。"
    "翻訳結果は後ほど提供されるので、それを見てプランを更新してください。"
    "最終的なプランが完成したら、ユーザーに提示してください。全ての作業が完了したと思ったら、最後に「TASK_COMPLETE」とだけ述べてください。"
    "もし以前に翻訳を依頼し、その結果がサブタスク結果として提供されている場合は、それを利用してプランニングを進めてください。"
    "{sub_task_result_info}" # サブタスク結果の情報を挿入するプレースホルダー
)

def travel_planner_agent_node(state: HierarchicalAgentState):
    print("\n--- Executing Travel Planner Agent ---")
    current_task_desc = state.get("task_description", "")
    sub_task_result = state.get("sub_task_result")
    
    sub_task_result_info = ""
    if sub_task_result:
        print(f"Travel Planner: Received sub_task_result: {sub_task_result}")
        sub_task_result_info = f"以前依頼した翻訳の結果: 「{sub_task_result}」"
        # サブタスク結果を使ったのでクリアする
        # state['sub_task_result'] = None # ノード内でstateを直接変更するのは推奨されない。代わりに辞書を返す
        
    # プロンプトにサブタスク結果情報を埋め込む
    formatted_prompt = travel_planner_prompt.format(sub_task_result_info=sub_task_result_info)
    
    # create_agent_node のロジックを一部展開して使用
    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(content=formatted_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])
    agent_runnable = prompt_template | llm
    response_message = agent_runnable.invoke({"messages": state['messages'], "sub_task_result_info": sub_task_result_info })
    print(f"Travel Planner Response: {response_message.content}")

    # 返却するStateの更新。サブタスク結果は消費したのでクリアする意図を込める。
    # ただし、stateの直接変更ではなく、更新辞書で返す。
    update_dict = {"messages": [response_message], "sub_task_result": None} 
    
    if "TRANSLATE:" in response_message.content:
        parts = response_message.content.split("TRANSLATE:", 1)
        sub_task_desc = parts[1].strip()
        print(f"Travel Planner: Identified sub-task: Translate '{sub_task_desc}'")
        update_dict["sub_task_description"] = sub_task_desc
        update_dict["original_requester"] = "TravelPlanner" # 戻り先を明示
        # next_agent はスーパーバイザーが決めるのでここでは設定しない
    elif "TASK_COMPLETE" in response_message.content:
        print("Travel Planner: Task marked as complete.")
        # next_agent はスーパーバイザーが決める (ENDになるはず)
        
    return update_dict

translator_agent_prompt = "あなたは翻訳家です。与えられたテキストを英語に翻訳してください。翻訳結果のみを返してください。"
translator_agent = create_agent_node(llm, translator_agent_prompt) # 入力はstate['messages']の最後を想定

def translator_agent_node_hierarchical(state: HierarchicalAgentState):
    print("\n--- Executing Translator Agent ---")
    # サブタスクの内容は sub_task_description にあると想定
    text_to_translate = state.get("sub_task_description")
    if not text_to_translate:
        print("Translator: No text to translate in sub_task_description.")
        return {"messages": [AIMessage(content="翻訳対象のテキストが指定されていません。")], "sub_task_result": "Error: No text provided"}

    # create_agent_node の内部ロジックと同様だが、入力メッセージを sub_task_description から作成
    # このエージェントは会話履歴ではなく、単一の指示で動く想定
    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(content=translator_agent_prompt),
        HumanMessage(content=f"以下のテキストを翻訳してください: {text_to_translate}"),
    ])
    agent_runnable = prompt_template | llm
    response_message = agent_runnable.invoke({}) # このプロンプトは入力メッセージを直接使わない
    print(f"Translator Response: {response_message.content}")
    
    # 結果を sub_task_result に格納し、呼び出し元（TravelPlanner）に戻す準備
    return {
        "messages": [response_message], # 翻訳エージェント自身の発言として記録
        "sub_task_result": response_message.content.strip(),
        "sub_task_description": None # 消費したのでクリア
    }

hierarchical_agent_names = ["TravelPlanner", "Translator"] # スーパーバイザーが知っているエージェント

# --- スーパーバイザー定義 (階層構造対応) ---
supervisor_prompt_hierarchical = (
    "あなたは高度なAIオーケストレーターです。メインタスクとサブタスクの進行を管理します。"
    "メインエージェント: TravelPlanner, サブエージェント: Translator."
    "現在の会話履歴とエージェントの状態を考慮して、次に実行すべきエージェントを決定してください。"
    "- TravelPlannerが「TRANSLATE:[テキスト]」という形式でメッセージを出力した場合、それは翻訳サブタスクの要求です。次にTranslatorを呼び出してください。"
    "- Translatorが作業を完了した場合（sub_task_resultに結果がある場合）、次にTravelPlannerを呼び出して結果を渡してください。original_requesterも参考にしてください。"
    "- TravelPlannerが「TASK_COMPLETE」と報告した場合、全作業完了なのでENDとしてください。"
    "- 通常はTravelPlannerに処理を促してください。"
    "- 入力メッセージ: {{#each messages}}{{this.type}}: {{this.content}}\n{{/each}}"
    "- 現在のタスク記述: {{task_description}}"
    "- サブタスク記述: {{sub_task_description}}"
    "- サブタスク結果: {{sub_task_result}}"
    "- 元の要求元: {{original_requester}}"
    "選択肢: TravelPlanner, Translator, END。名前のみを述べてください。"
)
# create_supervisor_node に渡すプロンプト
supervisor_prompt_for_helper_q4 = (
    "あなたは高度なAIオーケストレーターです。メインタスクとサブタスクの進行を管理します。"
    "メインエージェント: TravelPlanner, サブエージェント: Translator."
    "ユーザーの初期リクエストは task_description で確認できます。"
    "現在の会話履歴とエージェントの状態（sub_task_description, sub_task_result, original_requesterなど）をよく読んでください。"
    "判断基準:
    1. TravelPlannerからの最新メッセージに「TRANSLATE:」が含まれていれば、次は'Translator'です。
    2. stateの sub_task_result に結果があり、original_requester が 'TravelPlanner' であれば、次は'TravelPlanner'です。
    3. TravelPlannerからの最新メッセージに「TASK_COMPLETE」が含まれていれば、次は'END'です。
    4. 上記以外で、会話の文脈からTravelPlannerが続けるべきなら'TravelPlanner'です。
    5. それ以外、または判断に迷う場合は、'TravelPlanner'に処理を促してください（最終判断はTravelPlannerが行う）。"
    "次に実行すべきエージェントの名前（TravelPlanner, Translator, ENDのいずれか）のみを述べてください。"
    "利用可能なエージェントオプション: {agent_options}" # ヘルパーが埋める
)

supervisor_agent_q4 = create_supervisor_node(llm, hierarchical_agent_names, supervisor_prompt_for_helper_q4)

# --- ルーター関数 (階層スーパーバイザーの決定に基づく) ---
def hierarchical_router(state: HierarchicalAgentState):
    print(f"\n--- Hierarchical Router ---")
    # スーパーバイザーが next_agent を設定するが、ここでは追加ロジックも入れられる
    # 例えば、TravelPlannerがTRANSLATEを要求したら、強制的にTranslatorに向けるなど。
    # 今回はcreate_supervisor_nodeのLLM判断に任せる。
    next_agent_from_supervisor = state.get("next_agent")
    print(f"Supervisor decided next agent: {next_agent_from_supervisor}")

    # TravelPlanner が翻訳を要求した場合の遷移ロジック (LLMの判断を補助/上書きも可能)
    # last_message = state["messages"][-1] if state["messages"] else None
    # if isinstance(last_message, AIMessage) and "TRANSLATE:" in last_message.content:
    #     print("Router: Detected TRANSLATE request, overriding to Translator if not already set.")
    #     return "Translator"
    # if state.get("sub_task_result") and state.get("original_requester") == "TravelPlanner":
    #     print("Router: Detected sub_task_result for TravelPlanner, overriding to TravelPlanner if not already set.")
    #     return "TravelPlanner"

    if next_agent_from_supervisor in hierarchical_agent_names:
        return next_agent_from_supervisor
    elif next_agent_from_supervisor == "END":
        return END
    else:
        print(f"Warning: Unknown next_agent '{next_agent_from_supervisor}' from supervisor. Defaulting to TravelPlanner.")
        return "TravelPlanner" # 安全策としてメインエージェントに戻す

# --- グラフ構築 ---
workflow_q4 = ____(HierarchicalAgentState)

workflow_q4.add_node("Supervisor", supervisor_agent_q4)
workflow_q4.add_node("TravelPlanner", travel_planner_agent_node)
workflow_q4.add_node("Translator", translator_agent_node_hierarchical)

workflow_q4.set_entry_point("____") # Supervisor

workflow_q4.add_conditional_edges(
    "Supervisor",
    ____, # hierarchical_router
    {
        "TravelPlanner": "TravelPlanner",
        "Translator": "Translator",
        END: END
    }
)

# 各エージェントの処理が終わったら、再度スーパーバイザーに判断を仰ぐ
workflow_q4.add_edge("TravelPlanner", "____") # Supervisor
workflow_q4.add_edge("Translator", "____")  # Supervisor

app_q4 = workflow_q4.____() # compile

# --- グラフの実行と結果表示 ---
initial_task_q4 = "沖縄旅行を計画しています。航空券とホテルを探してください。あと、このレビュー「숙소가 깨끗하고 좋았지만, 바다 전망이 아니어서 아쉬웠어요. 하지만 조식은 훌륭했습니다!」を参考にしたいので、内容を教えてください。"
inputs_q4 = HierarchicalAgentState(
    messages=[HumanMessage(content=initial_task_q4)],
    task_description=initial_task_q4,
    next_agent=None, # 初期値
    original_requester=None,
    sub_task_description=None,
    sub_task_result=None
)

print(f"\n--- Running Graph for Hierarchical Task Q4: {initial_task_q4[:50]}... ---")
for s_chunk in app_q4.stream(inputs_q4, {"recursion_limit": 15}):
    node_name = list(s_chunk.keys())[0]
    print(f"\nOutput from node: {node_name}")
    content = s_chunk[node_name]
    if isinstance(content, dict):
        if 'messages' in content and content['messages']:
            latest_message = content['messages'][-1]
            print(f"  Message: {type(latest_message).__name__}: {latest_message.content[:100]}...")
        if 'next_agent' in content:
             print(f"  Next agent decided: {content['next_agent']}")
        if 'sub_task_result' in content and content['sub_task_result'] is not None:
            print(f"  Sub-task result updated: {content['sub_task_result']}")
    else:
        print(f"  Content: {content}")
    print("---")

final_state_q4 = app_q4.invoke(inputs_q4, {"recursion_limit": 15})
print("\n--- Final State Q4 ---")
for msg_idx, msg in enumerate(final_state_q4['messages']):
    print(f"[{msg_idx}] {type(msg).__name__}: {msg.content}")
print(f"Final task_description: {final_state_q4.get('task_description')}")
print(f"Final next_agent: {final_state_q4.get('next_agent')}")
print(f"Final original_requester: {final_state_q4.get('original_requester')}")
print(f"Final sub_task_description: {final_state_q4.get('sub_task_description')}")
print(f"Final sub_task_result: {final_state_q4.get('sub_task_result')}")


<details><summary>解答004</summary>

``````python
# 解答004
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessage # 追加
from typing import Sequence, TypedDict, Annotated, Optional
import operator

# --- 状態定義の更新 (サブタスク結果と元のタスク情報を保持) ---
class HierarchicalAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    next_agent: Optional[str]
    task_description: Optional[str]
    original_requester: Optional[str] # サブタスク完了後に戻るべきエージェント
    sub_task_description: Optional[str] # サブタスクの内容
    sub_task_result: Optional[str] # サブタスクの実行結果

# --- エージェント定義 ---
travel_planner_prompt_template_str = (
    "あなたは旅行プランナーです。ユーザーの要望に基づいて旅行プランを作成します。"
    "必要に応じて、外国語のレビューを翻訳するために「TRANSLATE:[レビューテキスト]」という形式で翻訳リクエストを生成できます。"
    "翻訳結果は後ほど提供されるので、それを見てプランを更新してください。"
    "最終的なプランが完成したら、ユーザーに提示してください。全ての作業が完了したと思ったら、最後に「TASK_COMPLETE」とだけ述べてください。"
    "もし以前に翻訳を依頼し、その結果がサブタスク結果として提供されている場合は、それを利用してプランニングを進めてください。\n"
    "{sub_task_result_info}" # サブタスク結果の情報を挿入するプレースホルダー
)

def travel_planner_agent_node(state: HierarchicalAgentState):
    print("\n--- Executing Travel Planner Agent ---")
    sub_task_result = state.get("sub_task_result")
    
    sub_task_result_info_str = ""
    if sub_task_result:
        print(f"Travel Planner: Received sub_task_result: {sub_task_result}")
        sub_task_result_info_str = f"以前依頼した翻訳の結果: 「{sub_task_result}」"
        
    current_system_prompt = travel_planner_prompt_template_str.format(sub_task_result_info=sub_task_result_info_str)
    
    prompt_for_llm = ChatPromptTemplate.from_messages([
        SystemMessage(content=current_system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])
    agent_runnable = prompt_for_llm | llm
    # runnableに渡す辞書から、プロンプトで使われないキーを削除 (MessagesPlaceholderが厳密なため)
    invoke_payload = {"messages": state['messages']}
    if "sub_task_result_info" in current_system_prompt: # 可変部分なので実際には使われないが念のため
        invoke_payload["sub_task_result_info"] = sub_task_result_info_str

    response_message = agent_runnable.invoke(invoke_payload)
    print(f"Travel Planner Response: {response_message.content}")

    update_dict = {"messages": [response_message], "sub_task_result": None} 
    
    if "TRANSLATE:" in response_message.content:
        parts = response_message.content.split("TRANSLATE:", 1)
        sub_task_desc = parts[1].strip()
        # もしサブタスク記述が長文応答の一部なら、適切に抽出するロジックが必要
        # 例: 「TRANSLATE:[レビューテキスト]」 のような明確な区切りがある場合
        # ここでは単純化のため、TRANSLATE: 以降全てをサブタスク記述とする
        print(f"Travel Planner: Identified sub-task: Translate '{sub_task_desc[:100]}...' ")
        update_dict["sub_task_description"] = sub_task_desc
        update_dict["original_requester"] = "TravelPlanner"
    elif "TASK_COMPLETE" in response_message.content:
        print("Travel Planner: Task marked as complete.")
        
    return update_dict

translator_agent_system_prompt = "あなたは翻訳家です。与えられたテキストを英語に翻訳してください。翻訳結果のみを返してください。"

def translator_agent_node_hierarchical(state: HierarchicalAgentState):
    print("\n--- Executing Translator Agent ---")
    text_to_translate = state.get("sub_task_description")
    if not text_to_translate:
        print("Translator: No text to translate in sub_task_description.")
        # エラーメッセージを sub_task_result に設定して返す
        ai_error_msg = AIMessage(content="翻訳対象のテキストが指定されていません。サブタスクを中止します。")
        return {"messages": [ai_error_msg], "sub_task_result": "Error: No text provided for translation.", "sub_task_description": None}

    print(f"Translator: Translating text: {text_to_translate[:100]}...")
    # create_agent_nodeの内部ロジックと似ているが、入力はsub_task_description固定
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(content=translator_agent_system_prompt),
        HumanMessage(content=text_to_translate) # 翻訳対象をHumanMessageとして渡す
    ])
    translator_runnable = prompt | llm
    response_message = translator_runnable.invoke({}) # このプロンプトはstateを直接参照しない
    print(f"Translator Response: {response_message.content}")
    
    return {
        "messages": [response_message], 
        "sub_task_result": response_message.content.strip(),
        "sub_task_description": None 
    }

hierarchical_agent_names_q4 = ["TravelPlanner", "Translator"]

supervisor_prompt_for_helper_q4 = (
    "あなたは高度なAIオーケストレーターです。メインタスクとサブタスクの進行を管理します。"
    "メインエージェント: TravelPlanner, サブエージェント: Translator."
    "ユーザーの初期リクエストは task_description で確認できます。"
    "現在の会話履歴とエージェントの状態（sub_task_description, sub_task_result, original_requesterなど）をよく読んでください。"
    "判断基準:\n"
    "1. stateの sub_task_description に翻訳すべきテキストがセットされており、original_requester が 'TravelPlanner' であれば、次は'Translator'です。\n"
    "2. stateの sub_task_result に翻訳結果がセットされており、original_requester が 'TravelPlanner' であれば、次は'TravelPlanner'です。\n"
    "3. TravelPlannerからの最新メッセージに「TASK_COMPLETE」が含まれていれば、次は'END'です。\n"
    "4. 上記以外で、会話の文脈からTravelPlannerが続けるべきなら'TravelPlanner'です。\n"
    "5. それ以外、または判断に迷う場合は、'TravelPlanner'に処理を促してください。"
    "次に実行すべきエージェントの名前（TravelPlanner, Translator, ENDのいずれか）のみを述べてください。"
    "利用可能なエージェントオプション: {agent_options}"
)
supervisor_agent_q4 = create_supervisor_node(llm, hierarchical_agent_names_q4, supervisor_prompt_for_helper_q4)

def hierarchical_router(state: HierarchicalAgentState):
    print(f"\n--- Hierarchical Router ---")
    next_agent_from_supervisor = state.get("next_agent")
    print(f"Supervisor decided next agent: {next_agent_from_supervisor}")
    
    # Supervisorの判断を優先するが、より明示的なルールベースの遷移も可能
    # 例えば、sub_task_description があり original_requester が設定されていれば Translator へ、など。
    # 今回は supervisor_agent_q4 のプロンプトに判断基準を記述しているので、その出力を信頼する。

    if next_agent_from_supervisor in hierarchical_agent_names_q4:
        return next_agent_from_supervisor
    elif next_agent_from_supervisor == "END":
        return END
    else:
        print(f"Warning: Unknown next_agent '{next_agent_from_supervisor}' from supervisor. Defaulting to TravelPlanner.")
        return "TravelPlanner"

# --- グラフ構築 ---
workflow_q4 = StateGraph(HierarchicalAgentState)

workflow_q4.add_node("Supervisor", supervisor_agent_q4)
workflow_q4.add_node("TravelPlanner", travel_planner_agent_node)
workflow_q4.add_node("Translator", translator_agent_node_hierarchical)

workflow_q4.set_entry_point("Supervisor")

workflow_q4.add_conditional_edges(
    "Supervisor",
    hierarchical_router,
    {
        "TravelPlanner": "TravelPlanner",
        "Translator": "Translator",
        END: END
    }
)

workflow_q4.add_edge("TravelPlanner", "Supervisor")
workflow_q4.add_edge("Translator", "Supervisor")

app_q4 = workflow_q4.compile()

# --- グラフの実行と結果表示 (変更なし) ---
initial_task_q4 = "沖縄旅行を計画しています。航空券とホテルを探してください。あと、このレビュー「숙소가 깨끗하고 좋았지만, 바다 전망이 아니어서 아쉬웠어요. 하지만 조식은 훌륭했습니다!」を参考にしたいので、内容を教えてください。"
inputs_q4 = HierarchicalAgentState(
    messages=[HumanMessage(content=initial_task_q4)],
    task_description=initial_task_q4,
    next_agent=None, 
    original_requester=None,
    sub_task_description=None,
    sub_task_result=None
)

print(f"\n--- Running Graph for Hierarchical Task Q4: {initial_task_q4[:50]}... ---")
for s_chunk in app_q4.stream(inputs_q4, {"recursion_limit": 15}):
    node_name = list(s_chunk.keys())[0]
    print(f"\nOutput from node: {node_name}")
    content = s_chunk[node_name]
    if isinstance(content, dict):
        if 'messages' in content and content['messages']:
            latest_message = content['messages'][-1]
            print(f"  Message: {type(latest_message).__name__}: {latest_message.content[:100]}...")
        if 'next_agent' in content and content['next_agent'] is not None:
             print(f"  Next agent decided: {content['next_agent']}")
        if 'sub_task_result' in content and content['sub_task_result'] is not None:
            print(f"  Sub-task result updated: {content['sub_task_result']}")
        if 'sub_task_description' in content and content['sub_task_description'] is not None:
            print(f"  Sub-task description updated: {content['sub_task_description'][:100]}...")
    else:
        print(f"  Content: {content}")
    print("---")

final_state_q4 = app_q4.invoke(inputs_q4, {"recursion_limit": 15})
print("\n--- Final State Q4 ---")
for msg_idx, msg in enumerate(final_state_q4['messages']):
    print(f"[{msg_idx}] {type(msg).__name__}: {msg.content}")
print(f"Final task_description: {final_state_q4.get('task_description')}")
print(f"Final next_agent: {final_state_q4.get('next_agent')}")
print(f"Final original_requester: {final_state_q4.get('original_requester')}")
print(f"Final sub_task_description: {final_state_q4.get('sub_task_description')}")
print(f"Final sub_task_result: {final_state_q4.get('sub_task_result')}")
``````
</details>

<details><summary>解説004</summary>

#### この問題のポイント
*   **学習内容:**
    *   階層的なタスク処理のシミュレーション：主担当エージェント（TravelPlanner）がサブタスク（翻訳）を特定し、スーパーバイザーを介して専門のサブエージェント（Translator）に処理を委任する方法。
    *   状態 (`HierarchicalAgentState`) の拡張：サブタスクの情報（`sub_task_description`、`sub_task_result`）や、サブタスク完了後に処理を戻すべきエージェント（`original_requester`）を状態に保持する方法。
    *   エージェントのプロンプト調整：主担当エージェントがサブタスクを要求する特定のキーワード（例：「TRANSLATE:」）を出力し、また、提供されたサブタスクの結果を自身の処理に組み込むようにする。
    *   スーパーバイザーの役割拡張：サブタスク要求の検知、適切なサブエージェントへのルーティング、サブタスク完了後の結果の元の要求元への引き渡し、そして最終的なタスク完了の判断など、より高度なオーケストレーションを行う。
*   **コード解説:**
    *   **`HierarchicalAgentState`:**
        *   `original_requester`: サブタスクを依頼したエージェントの名前を保持。翻訳完了後、TravelPlannerに戻すために使用。
        *   `sub_task_description`: TravelPlannerがTranslatorに依頼したい翻訳対象テキストなどを保持。
        *   `sub_task_result`: Translatorの翻訳結果を保持し、TravelPlannerが利用できるようにする。
    *   **`travel_planner_agent_node`:**
        *   プロンプトが更新され、`sub_task_result` があればそれを考慮してプランニングするよう指示。また、「TRANSLATE:[テキスト]」という形式で翻訳リクエストを生成する能力を持つ。
        *   翻訳リクエストを検知すると、`sub_task_description` と `original_requester` をStateにセットする。
        *   「TASK_COMPLETE」というキーワードでタスク完了を示す。
        *   サブタスク結果 (`sub_task_result`) を受け取ったら、それを消費したことを示すためにState更新で `None` に戻す。
    *   **`translator_agent_node_hierarchical`:**
        *   `state['sub_task_description']` から翻訳対象テキストを取得して翻訳を実行。
        *   結果を `state['sub_task_result']` に格納し、`sub_task_description` は消費済みとして `None` にする。
    *   **`supervisor_agent_q4` (スーパーバイザー):**
        *   プロンプト (`supervisor_prompt_for_helper_q4`) が大幅に拡張され、`sub_task_description` の存在（翻訳要求）、`sub_task_result` の存在（翻訳完了）、`original_requester` の情報、TravelPlannerからの「TASK_COMPLETE」報告などを総合的に判断して、次に `TravelPlanner`、`Translator`、または `END` のいずれにルーティングするかを決定するよう指示されている。
    *   **`hierarchical_router`:** スーパーバイザーの決定に従ってルーティングする。
    *   **グラフ構築:**
        *   `Supervisor` がエントリーポイント。
        *   `TravelPlanner` と `Translator` の両方とも、処理完了後は `Supervisor` に戻り、次の指示を仰ぐ。
    *   **実行例:** 韓国語のレビューを含む旅行計画の相談。TravelPlannerがレビュー翻訳を要求 → SupervisorがTranslatorへ → Translatorが翻訳 → SupervisorがTravelPlannerへ翻訳結果を渡す → TravelPlannerが計画を完成させTASK_COMPLETE → SupervisorがEND、という流れを想定。

この演習では、エージェントが動的にサブタスクを生成し、それを別の専門エージェントに委任し、その結果を再び受け取って主タスクを続行するという、より高度で柔軟なマルチエージェント連携のパターンを実装しています。状態管理とスーパーバイザーのインテリジェンスが、このような複雑なフローを実現する上での鍵となります。
---
</details>

## 第4章まとめ

この章では、マルチエージェントシステムを構築するための様々なアプローチを学びました。

1.  **基本的な2エージェント会話（問題001）**: 複数のエージェントが交互に情報をやり取りする基本形を実装しました。ルーター関数を使って会話の流れを制御しました。
2.  **スーパーバイザーによるタスク割り振り（問題002）**: スーパーバイザーエージェントがタスク内容を判断し、適切な専門ワーカーエージェントに処理を割り振る構造を学びました。これは、タスクに応じて動的に処理フローを変更する基本的なパターンです。
3.  **スーパーバイザーによる複数エージェントの逐次連携（問題003）**: より複雑なタスクに対応するため、スーパーバイザーが複数のエージェントの作業を順番に連携させる方法を実装しました。ワーカーエージェントの処理完了後にスーパーバイザーに制御を戻し、次のステップを判断させるループ構造が特徴です。
4.  **階層型エージェント構造（簡易版）（問題004）**: 主担当エージェントが処理中にサブタスクを発見し、それをスーパーバイザー経由で専門のサブエージェントに委任し、結果を受け取って主タスクを続行する、より高度な連携パターンを学びました。状態(`State`)にサブタスク関連の情報（要求元、タスク内容、結果）を持たせることで、このような階層的な処理を実現しました。

これらの演習を通じて、`StateGraph`における状態の設計、ノード（エージェント）の定義、そしてエッジ（特に条件付きエッジとルーター関数）を使った柔軟な制御フローの構築方法についての理解が深まったことでしょう。
マルチエージェントシステムは、複雑な問題を複数の専門家（エージェント）に分割して処理させることで、より高度で柔軟なAIアプリケーションを実現するための強力なパラダイムです。
LangGraphは、このようなシステムの構築を支援するための堅牢なフレームワークを提供します。